## Transform ASTER Level-2 Data into Tiled Northup Product

This notebook provides a protoype function to convert the Level-2 V4 ASTER Products from their archieved rotated map orientation trasnformation to a northup transformation tiled product based upon the HLS (MGRS) Grid.

The goals include:
1. Convert Rotated ASTER Data to Northup tiled product to support potential time series and mosaicing
2. Preserve input paramters (height, wideth, crs, pixel centers) in order to preserve alignment between northup 
3. Write transformed tifs as some output that can be viewed in a GIS to compare with input data

MGRS Grid is Read in from the [HLS Website](https://hls.gsfc.nasa.gov/products-description/tiling-system/)

As of 12/12/25 this method was tested on AST_07 for one acquisition all bands. Still need to try with the other Level-2 Products (Visable: AST_09, AST_09X, AST_07X) and (Thermal:  AST_09T, AST_08, AST_05).

To do:

1. Instead of writing outputs save in memory and process somehow into some usecase 

In [1]:
import geopandas as gpd
import os
import rioxarray as rxr
import earthaccess
from osgeo import gdal
import pandas as pd
import fiona
from rasterio.transform import from_origin
from rasterio.warp import Resampling
import numpy as np
from shapely.geometry import box
import matplotlib.pyplot as plt
import xarray as xr
import requests
import tempfile
import rasterio
import earthaccess
from dask.distributed import Client, LocalCluster
import dask.distributed

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
fiona.supported_drivers['KML'] = 'rw'

In [2]:
#Set working directory for outputs
os.chdir(r'C:\Users\nroberts\aster\data')

#Link to MGRS Grid KML
mgrs_path  = 'S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'

#aoi
aoi = gpd.read_file('railroad_valley.geojson')
bbox = tuple(list(aoi.total_bounds))

In [3]:
bbox = tuple(list(aoi.total_bounds))
bbox

(np.float64(-115.89358304108943),
 np.float64(38.31559085515974),
 np.float64(-115.33911006153537),
 np.float64(38.5826536407757))

In [4]:
temporal = ("2000-01-05T00:00:00", "2000-12-31T23:59:59")

In [5]:
results = earthaccess.search_data(
    concept_id='C3306877498-LPCLOUD',
    bounding_box=bbox,
    temporal=temporal,
    provider='LPCLOUD',
    cloud_cover=(0,5),
)
print(len(results))

pd.json_normalize(results)

10


,size,meta.concept-type,meta.concept-id,meta.revision-id,meta.native-id,meta.collection-concept-id,meta.provider-id,meta.format,meta.revision-date,umm.TemporalExtent.SingleDateTime,umm.GranuleUR,umm.AdditionalAttributes,umm.MeasuredParameters,umm.SpatialExtent.HorizontalSpatialDomain.Geometry.GPolygons,umm.ProviderDates,umm.CollectionReference.ShortName,umm.CollectionReference.Version,umm.PGEVersionClass.PGEVersion,umm.RelatedUrls,umm.CloudCover,umm.InputGranules,umm.DataGranule.Identifiers,umm.DataGranule.DayNightFlag,umm.DataGranule.ProductionDateTime,umm.DataGranule.ArchiveAndDistributionInformation,umm.Platforms,umm.MetadataSpecification.URL,umm.MetadataSpecification.Name,umm.MetadataSpecification.Version
0,82.159832,granule,G3894750374-LPCLOUD,1,AST_07_00405012000190040_20251203152300,C3306877498-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-12-03T22:40:26.017Z,2000-05-01T19:00:40.72200Z,AST_07_00405012000190040_20251203152300,"[{'Name': 'Band01LocalGranuleID', 'Values': ['...",[{'ParameterName': 'Surface Reflectance - VNIR...,[{'Boundary': {'Points': [{'Longitude': -116.3...,"[{'Date': '2025-12-03T21:40:53.000Z', 'Type': ...",AST_07,004,Correct_VS:3.4,[{'URL': 'https://doi.org/10.5067/ASTER/AST_07...,0,"[AST_07_00405012000190040_20251203152300.hdf, ...",[{'Identifier': 'AST_07_00405012000190040_2025...,Day,2025-12-03T21:40:53.000Z,[{'Name': 'AST_07_00405012000190040_2025120315...,"[{'ShortName': 'TERRA', 'Instruments': [{'Shor...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
1,83.580978,granule,G3894860802-LPCLOUD,1,AST_07_00405012000190049_20251203163451,C3306877498-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-12-03T23:39:26.102Z,2000-05-01T19:00:49.55800Z,AST_07_00405012000190049_20251203163451,"[{'Name': 'Band01LocalGranuleID', 'Values': ['...",[{'ParameterName': 'Surface Reflectance - VNIR...,[{'Boundary': {'Points': [{'Longitude': -116.4...,"[{'Date': '2025-12-03T22:49:28.000Z', 'Type': ...",AST_07,004,Correct_VS:3.4,[{'URL': 'https://doi.org/10.5067/ASTER/AST_07...,0,"[AST_07_00405012000190049_20251203163451.hdf, ...",[{'Identifier': 'AST_07_00405012000190049_2025...,Day,2025-12-03T22:49:28.000Z,[{'Name': 'AST_07_00405012000190049_2025120316...,"[{'ShortName': 'TERRA', 'Instruments': [{'Shor...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
2,85.739986,granule,G3381036200-LPCLOUD,2,AST_07_00406022000185958_20250124215439,C3306877498-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-04-18T03:24:09.328Z,2000-06-02T18:59:58.142000Z,AST_07_00406022000185958_20250124215439,"[{'Name': 'Band01LocalGranuleID', 'Values': ['...",[{'ParameterName': 'Surface Reflectance - VNIR...,[{'Boundary': {'Points': [{'Longitude': -116.2...,"[{'Date': '2025-01-25T04:12:23.000Z', 'Type': ...",AST_07,004,Correct_VS:3.4,[{'URL': 'https://doi.org/10.5067/ASTER/AST_07...,2,"[AST_07_00406022000185958_20250124215439.hdf, ...",[{'Identifier': 'AST_07_00406022000185958_2025...,Day,2025-01-25T04:12:23.000Z,[{'Name': 'AST_07_00406022000185958_2025012421...,"[{'ShortName': 'TERRA', 'Instruments': [{'Shor...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
3,86.793748,granule,G3381036159-LPCLOUD,2,AST_07_00406022000190006_20250124215740,C3306877498-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-04-18T03:19:09.975Z,2000-06-02T19:00:06.982000Z,AST_07_00406022000190006_20250124215740,"[{'Name': 'Band01LocalGranuleID', 'Values': ['...",[{'ParameterName': 'Surface Reflectance - VNIR...,[{'Boundary': {'Points': [{'Longitude': -116.3...,"[{'Date': '2025-01-25T04:14:26.000Z', 'Type': ...",AST_07,004,Correct_VS:3.4,[{'URL': 'https://doi.org/10.5067/ASTER/AST_07...,2,"[AST_07_00406022000190006_20250124215740.hdf, ...",[{'Identifier': 'AST_07_00406022000190006_2025...,Day,2025-01-25T04:14:26.000Z,[{'Name': 'AST_07_00406022000190006_2025012421...,"[{'ShortName': 'TERRA', 'Instruments': [{'Shor...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
4,88.540433,granule,G338112

In [6]:
urls = [granule.data_links() for granule in results]
urls = [item for sublist in urls for item in sublist]
urls

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B04.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B05.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B06.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B07.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012000190040_20251203152301_SRF_SWIR_B08.tif',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/AST_07.004/AST_07_00405012000190040_20251203152300/AST_07_00405012

In [7]:
def setup_dask_environment():
    """
    Passes RIO environment variables to dask workers for authentication.
    """
    import os
    import rasterio

    global env
    env = rasterio.Env(
        GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
        GDAL_HTTP_COOKIEFILE=os.path.expanduser("~/cookies.txt"),
        GDAL_HTTP_COOKIEJAR=os.path.expanduser("~/cookies.txt"),
        GDAL_HTTP_MAX_RETRY="10",
        GDAL_HTTP_RETRY_DELAY="0.5",
    )
    env.__enter__()

In [8]:
#Read in MGRS (HLS) Grid from the HLS Website and return as geopandas dataframe
def load_mgrs(mgrs_path, aoi):

    #response = requests.get(mgrs_url)

    #with tempfile.NamedTemporaryFile(suffix='.kml', delete=False) as tmp:
    #    tmp.write(response.content)
    #    tmp_path = tmp.name

    mgrs_grid = gpd.read_file(mgrs_path, driver = 'KML')
    mgrs_grid = mgrs_grid.set_crs('EPSG:4326')
    mgrs_grid = gpd.sjoin(mgrs_grid, aoi, predicate='intersects')
    mgrs_grid = mgrs_grid.drop(['index_right'], axis=1)
    return mgrs_grid

In [9]:
#Read in Source (Rotated) Data and return required variables
def read_rotated(input_file, 
                nodata = 0):
    
    rotated = rxr.open_rasterio(input_file, mask_and_scale=False, chunks='auto')
    rotated = rotated.rio.write_nodata(nodata)
    rotated_transform = rotated.rio.transform()
    rotated_crs = rotated.rio.crs

    #return rotated xarray, rotated transformation, rotated_crs, rotated_bounds

    return rotated, rotated_transform, rotated_crs

In [10]:
"""Calculate true bounds of the rotated tif, it does not work to just use the bounds of the rotated
data as GIS flips the bounds north because data is assumed not to be rotated by GIS and by
geopandas
"""
def get_rotated_bounds(rotated, rotated_crs):
    rotated_bounds = rasterio.transform.array_bounds(rotated.rio.width, rotated.rio.height, rotated.rio.transform())
    minx, miny, maxx, maxy = rotated_bounds
    geom = box(minx, miny, maxx, maxy) 
    
    gdf_bounds = gpd.GeoDataFrame({'geometry': [geom]}, crs = rotated_crs)
    gdf_bounds = gdf_bounds.to_crs('epsg:4326')

    return gdf_bounds

In [11]:
#Get intesecting MGRS tiles from MGRS Grid and return the tile bounds
def get_mgrs_tile(mgrs_grid, gdf_bounds, rotated_crs):
    mgrs_tile = gpd.sjoin(mgrs_grid, gdf_bounds, predicate='intersects')



    mgrs_tile = mgrs_tile.to_crs(rotated_crs)
    mgrs_tile['minx'] = mgrs_tile.bounds['minx']
    mgrs_tile['miny'] = mgrs_tile.bounds['miny']
    mgrs_tile['maxx'] = mgrs_tile.bounds['maxx']
    mgrs_tile['maxy'] = mgrs_tile.bounds['maxy']
    
    return mgrs_tile


In [12]:
"""
This is where we create the northup tiled products.

1. Iterate over intersecting tiles geodataframe so we get a tile for each part of the source tif
covered.
2. Get bounds for the bounding box of the MGRS Tile.
3. Calculate the X, Y Resolution we can't just use the resolution from the rotated transformation
because it is several decimal points different then the published resolution of 15, 30, and 90 meters
for VNIR, SWIR and TIR.
4. Calculate shape height.
5. Create new transformation for north up
6. Reproject rotated data using north up transform and with and height for mgrs_tiled data
7. Write out ouputs
8. Add coords that might be useful to create a xarray dataset later (Band Name and MGRS Tile ID)
"""
def make_tiled(rotated, rotated_transform, mgrs_tile, rotated_crs, tif, aoi):


    for i, row in mgrs_tile.iterrows():
   
        minx = row['minx']
        maxy = row['maxy']
        maxx = row['maxx']
        miny = row['miny']


    
        res_x = np.sqrt(rotated_transform.a**2 + rotated_transform.b**2)
        res_y = np.sqrt(rotated_transform.d**2 + rotated_transform.e**2)

        out_with = int((maxx - minx)/res_x)
        out_height = int((maxy - miny)/res_y)

        north_up = from_origin(minx, maxy, res_x, res_y)

        mgrs_aster = rotated.rio.reproject(rotated.rio.crs, shape = (out_height, out_with), transform = north_up, resampling = Resampling.bilinear)
        tname = row['Name']
        output_file = tif.split('/')[6][:-4]
        if 'QA' in output_file:
            band_name = '_'.join(output_file.split('_')[5:8])
        else:
            band_name = '_'.join(output_file.split('_')[5:7])
        
        mgrs_aster = mgrs_aster.squeeze('band', drop = True)
        mgrs_aster = mgrs_aster.rename(band_name)
        mgrs_aster = mgrs_aster.assign_coords(tile_id = tname)

        aoi_reproj = aoi.to_crs(mgrs_aster.rio.crs)
        clipped = mgrs_aster.rio.clip(aoi_reproj.geometry.values, aoi_reproj.crs, all_touched=True)
        clipped.rio.to_raster(f'{output_file}_{tname}_{'bounds'}.tif')

    return clipped

In [13]:
# Initialize Dask Cluster
client = dask.distributed.Client()

# Setup Dask Environment (GDAL Configs)
client.run(setup_dask_environment)

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 31.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54496,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:54518,Total threads: 3
Dashboard: http://127.0.0.1:54523/status,Memory: 7.92 GiB
Nanny: tcp://127.0.0.1:54499,


In [14]:
mgrs_grid = load_mgrs(mgrs_path, aoi)

def process_file(tif, mgrs_grid, aoi):
    rotated, rotated_transform, rotated_crs, = read_rotated(tif)
    rotated_bounds = get_rotated_bounds(rotated, rotated_crs)
    mgrs_tile = get_mgrs_tile(mgrs_grid, rotated_bounds, rotated_crs)

    make_tiled(rotated, rotated_transform, mgrs_tile, rotated_crs,tif, aoi)

lazy_results = [dask.delayed(process_file)(f, mgrs_grid, aoi) for f in urls]

# Compute the results in parallel
results = dask.compute(*lazy_results)

c:\Users\nroberts\AppData\Local\miniforge3\envs\lpdaac\Lib\site-packages\pyogrio\geopandas.py:275: UserWarning: More than one layer found in 'S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml': 'Features' (default), 'Info'. Specify layer parameter to avoid this warning.
  result = read_func(


In [15]:
client.close()